# Test of HiPace300 Pumps on Transfer and Depo Chamber

## Equipment Configuration:
- **HiPace300 Pumps**: Connected to Transfer and Deposition Chambers
- **HiScroll12**: One unit connected as backing pump
- **Pressure Sensors**: Two total sensors connected to both Omnicontrol 300 units

## Test Overview:
This notebook contains tests and monitoring for the HiPace300 turbo molecular pump system with integrated pressure monitoring and control capabilities.

## 1. Import Required Libraries and Setup


In [1]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.hiscroll12.hiscroll12 import HiScroll12
from devices.pfeiffer.hipacebus import HiPace300Bus

## 2. Setup Shared Logging System

In [2]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"hipace_Transfer_Depo_test_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("Pump Locker Test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

2025-08-14 13:17:59 | INFO | Pump Locker Test system initialized with shared logging


Repository root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Shared logs will be saved to: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\hipace_Transfer_Depo_test_20250814_131759.log


# 3. Set COM-Ports & Adresses

In [3]:
# Device configuration dictionaries
# Modify these COM ports and addresses according to your actual hardware setup

HiScroll12_dict = {
    'com_port': 'COM19',
    'device_address': 2,
    'device_type': 'HiScroll12',
    'description': 'Backing pump for HiPace300 systems'
}

HiPace300_Transfer_dict = {
    'com_port': 'COM36',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address':1,
    'gauge1_address': 122,
    'device_type': 'HiPace300',
    'description': 'Transfer chamber turbo pump'
}

HiPace300_Depo_dict = {
    'com_port': 'COM37',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address':1,
    'gauge1_address': 122,
    'device_type': 'HiPace300', 
    'description': 'Deposition chamber turbo pump'
}

# Print configuration summary
print("Device Configuration Summary:")
print("=" * 40)
print(f"HiScroll12 (Backing pump):")
print(f"  COM Port: {HiScroll12_dict['com_port']}")
print(f"  Address: {HiScroll12_dict['device_address']}")
print(f"  Description: {HiScroll12_dict['description']}")

print(f"\nHiPace300 Transfer:")
print(f"  COM Port: {HiPace300_Transfer_dict['com_port']}")
print(f"  Address: {HiPace300_Transfer_dict['device_address']}")
print(f"  Description: {HiPace300_Transfer_dict['description']}")

print(f"\nHiPace300 Depo:")
print(f"  COM Port: {HiPace300_Depo_dict['com_port']}")
print(f"  Address: {HiPace300_Depo_dict['device_address']}")
print(f"  Description: {HiPace300_Depo_dict['description']}")

print("\n✅ Device dictionaries configured successfully")

Device Configuration Summary:
HiScroll12 (Backing pump):
  COM Port: COM19
  Address: 2
  Description: Backing pump for HiPace300 systems

HiPace300 Transfer:
  COM Port: COM36
  Address: 101
  Description: Transfer chamber turbo pump

HiPace300 Depo:
  COM Port: COM37
  Address: 101
  Description: Deposition chamber turbo pump

✅ Device dictionaries configured successfully


# 4. Connect Devices

In [4]:
# Initialize and connect HiScroll12
hiscroll = HiScroll12(
    device_id="hiscroll12",
    port=HiScroll12_dict['com_port'],
    device_address=HiScroll12_dict['device_address'],
    logger=logger
)
hiscroll.connect()
print(f"✅ HiScroll12 connected on {HiScroll12_dict['com_port']}")

2025-08-14 13:18:11 | INFO | Connecting to Pfeiffer device hiscroll12 on COM19
2025-08-14 13:18:11 | INFO | Successfully connected to device at address 2


✅ HiScroll12 connected on COM19


In [5]:
# Initialize and connect HiPace300 Transfer
hipace_transfer = HiPace300Bus(
    device_id="hipace_transfer",
    port=HiPace300_Transfer_dict['com_port'],
    device_address=HiPace300_Transfer_dict['device_address'],
    omnicontrol_address=HiPace300_Transfer_dict['omnicontrol_address'],
    tc400_address=HiPace300_Transfer_dict['tc400_address'],
    gauge1_address=HiPace300_Transfer_dict['gauge1_address'],
    logger=logger
)

hipace_transfer.connect()
print(f"✅ HiPace300 Transfer connected on {HiPace300_Transfer_dict['com_port']}")

2025-08-14 13:18:12 | INFO | Connecting to Pfeiffer device hipace_transfer on COM36
2025-08-14 13:18:12 | INFO | Successfully connected to device at address 101


✅ HiPace300 Transfer connected on COM36


In [6]:
# Initialize and connect HiPace300 Depo
hipace_depo = HiPace300Bus(
    device_id="hipace_depo",
    port=HiPace300_Depo_dict['com_port'],
    device_address=HiPace300_Depo_dict['device_address'],
    omnicontrol_address=HiPace300_Depo_dict['omnicontrol_address'],
    tc400_address=HiPace300_Depo_dict['tc400_address'],
    gauge1_address=HiPace300_Depo_dict['gauge1_address'],
    logger=logger
)

hipace_depo.connect()
print(f"✅ HiPace300 Depo connected on {HiPace300_Depo_dict['com_port']}")

2025-08-14 13:18:14 | INFO | Connecting to Pfeiffer device hipace_depo on COM37
2025-08-14 13:18:14 | INFO | Successfully connected to device at address 101


✅ HiPace300 Depo connected on COM37


# 5. Debugging/Testing Connection manually

In [7]:
hiscroll.hk_monitor()
hipace_depo.hk_monitor()
hipace_transfer.hk_monitor()

2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Pump_Enabled=False//
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Standby_Mode=False//
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Speed_RPM=0//RPM
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Speed_Hz=0//Hz
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Temp_Motor=28//degC
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Temp_Electronics=33//degC
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Temp_Power_Stage=28//degC
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Drive_Current=0.0//A
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Drive_Voltage=399.92//V
2025-08-14 13:18:17 | INFO | hiscroll12//COM19//Drive_Power=0//W
2025-08-14 13:18:17 | INFO | hipace_depo//COM37//Pump_Station_Enabled=False//
2025-08-14 13:18:17 | INFO | hipace_depo//COM37//Standby_Mode=False//
2025-08-14 13:18:17 | INFO | hipace_depo//COM37//Motor_Pump_Enabled=False//
2025-08-14 13:18:17 | INFO | hipace_depo//COM37//Vent_Enabled=False//
2025-08-14 13:18:1

# 6. Start Housekeeping for All Pumps

In [8]:
# Start housekeeping for all pumps
hiscroll.start_housekeeping()
hipace_transfer.start_housekeeping()
hipace_depo.start_housekeeping()

print("✅ Housekeeping started for all pumps:")
print("  - HiScroll12")
print("  - HiPace300 Transfer")
print("  - HiPace300 Depo")

2025-08-14 13:18:38 | INFO | Using external logger - no additional file logging needed
2025-08-14 13:18:38 | INFO | Housekeeping worker started for hiscroll12
2025-08-14 13:18:38 | INFO | Housekeeping started (internal mode) - interval: 30.0s
2025-08-14 13:18:38 | INFO | Using external logger - no additional file logging needed
2025-08-14 13:18:38 | INFO | Housekeeping worker started for hipace_transfer
2025-08-14 13:18:38 | INFO | Housekeeping started (internal mode) - interval: 1.0s
2025-08-14 13:18:38 | INFO | Using external logger - no additional file logging needed
2025-08-14 13:18:38 | INFO | Housekeeping worker started for hipace_depo
2025-08-14 13:18:38 | INFO | Housekeeping started (internal mode) - interval: 1.0s


✅ Housekeeping started for all pumps:
  - HiScroll12
  - HiPace300 Transfer
  - HiPace300 Depo


In [ ]:
hiscroll.stop_housekeeping()
hipace_transfer.stop_housekeeping()
hipace_depo.stop_housekeeping()

In [10]:
hipace_depo.get_SensOnOff()

0

# 7. Live Plotting - Multi-Pump Visualization

In [11]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from collections import deque

class MultiPumpPlotter:
    def __init__(self, update_interval=2000):
        self.update_interval = update_interval
        
        # Data storage (keep last 100 points)
        self.timestamps = deque(maxlen=100)
        self.hiscroll_temp = deque(maxlen=100)
        self.hiscroll_power = deque(maxlen=100)
        
        self.hipace_t_temp = deque(maxlen=100)
        self.hipace_t_power = deque(maxlen=100)
        self.hipace_t_rpm = deque(maxlen=100)
        self.hipace_t_pressure = deque(maxlen=100)
        
        self.hipace_d_temp = deque(maxlen=100)
        self.hipace_d_power = deque(maxlen=100)
        self.hipace_d_rpm = deque(maxlen=100)
        self.hipace_d_pressure = deque(maxlen=100)
        
        # Create figure with 2x2 subplots
        self.fig, ((self.ax1, self.ax2), (self.ax3, self.ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    
    def update_plot(self, frame):
        try:
            current_time = datetime.now()
            self.timestamps.append(current_time)
            
            # HiScroll12 data
            self.hiscroll_temp.append(hiscroll.get_temp_motor())
            self.hiscroll_power.append(hiscroll.get_drive_power())
            
            # HiPace Transfer data
            self.hipace_t_temp.append(hipace_transfer.get_motor_temperature())
            self.hipace_t_power.append(hipace_transfer.get_drive_power())
            self.hipace_t_rpm.append(hipace_transfer.get_actual_speed_rpm())
            self.hipace_t_pressure.append(hipace_transfer.get_gauge_pressure())
            
            # HiPace Depo data
            self.hipace_d_temp.append(hipace_depo.get_motor_temperature())
            self.hipace_d_power.append(hipace_depo.get_drive_power())
            self.hipace_d_rpm.append(hipace_depo.get_actual_speed_rpm())
            self.hipace_d_pressure.append(hipace_depo.get_gauge_pressure())
            
        except Exception as e:
            logger.warning(f"Data collection error: {e}")
            return
        
        # Clear only the axes, not the entire figure
        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax4.clear()
        
        # Convert deques to lists for reliable plotting
        times = list(self.timestamps)
        hiscroll_temps = list(self.hiscroll_temp)
        hiscroll_powers = list(self.hiscroll_power)
        hipace_t_temps = list(self.hipace_t_temp)
        hipace_t_powers = list(self.hipace_t_power)
        hipace_t_rpms = list(self.hipace_t_rpm)
        hipace_t_pressures = list(self.hipace_t_pressure)
        hipace_d_temps = list(self.hipace_d_temp)
        hipace_d_powers = list(self.hipace_d_power)
        hipace_d_rpms = list(self.hipace_d_rpm)
        hipace_d_pressures = list(self.hipace_d_pressure)
        
        # Only plot if we have data
        if not times:
            return
        try:
            # Plot 1: Temperature & Power (HiScroll + HiPace)
            self.ax1.plot(times, hiscroll_temps, 'b-', label='HiScroll Temp', marker='o')
            self.ax1.plot(times, hipace_t_temps, 'r-', label='HiPace-T Temp', marker='s')
            self.ax1.plot(times, hipace_d_temps, 'g-', label='HiPace-D Temp', marker='^')
            #ax1_twin = self.ax1.twinx()
            #ax1_twin.plot(times, hiscroll_powers, 'b--', alpha=0.7, label='HiScroll Power')
            #ax1_twin.plot(times, hipace_t_powers, 'r--', alpha=0.7, label='HiPace-T Power')
            #ax1_twin.plot(times, hipace_d_powers, 'g--', alpha=0.7, label='HiPace-D Power')
            self.ax1.set_ylabel('Temperature (°C)')
            #ax1_twin.set_ylabel('Power (W)')
            self.ax1.set_title('Temperature & Power')
            self.ax1.legend(loc='upper left')
            #ax1_twin.legend(loc='upper right')
            self.ax1.grid(True)

            # Plot 2: HiPace Rotation Speed
            self.ax2.plot(times, hipace_t_rpms, 'r-', label='Transfer RPM', marker='s')
            self.ax2.plot(times, hipace_d_rpms, 'g-', label='Depo RPM', marker='^')
            self.ax2.set_ylabel('RPM')
            self.ax2.set_title('HiPace300 Rotation Speed')
            self.ax2.legend()
            self.ax2.grid(True)

            # Plot 3: Pressure (Log scale)
            self.ax3.plot(times, hipace_t_pressures, 'r-', label='Transfer Pressure', marker='s')
            self.ax3.plot(times, hipace_d_pressures, 'g-', label='Depo Pressure', marker='^')
            self.ax3.set_yscale('log')
            self.ax3.set_ylabel('Pressure (log scale)')
            self.ax3.set_title('Chamber Pressures')
            self.ax3.legend()
            self.ax3.grid(True)

            # Plot 4: Power Comparison
            self.ax4.plot(times, hiscroll_powers, 'b-', label='HiScroll Power', marker='o')
            self.ax4.plot(times, hipace_t_powers, 'r-', label='HiPace-T Power', marker='s')
            self.ax4.plot(times, hipace_d_powers, 'g-', label='HiPace-D Power', marker='^')
            self.ax4.set_ylabel('Power (W)')
            self.ax4.set_xlabel('Time')
            self.ax4.set_title('Power Consumption Comparison')
            self.ax4.legend()
            self.ax4.grid(True)

        except Exception as e:
            print("failed plotting")
        
        # Format all plots
        for ax in [self.ax1, self.ax2, self.ax3, self.ax4]:
            ax.tick_params(axis='x', rotation=45)
        
        self.fig.tight_layout()
    
    def start_live_plot(self):
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,
            cache_frame_data=False
        )

        plt.show()
        print("Live plotting started - Close window to stop")

        #return self.timestamps, self.hiscroll_temp, self.hipace_t_temp, self.hipace_d_temp, self.hiscroll_power, self.hipace_t_power, self.hipace_d_power
    
    def stop_live_plot(self):
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            print("Live plotting stopped.")
        else:
            print("No animation to stop.")

print("✅ Multi-pump plotter class ready")


✅ Multi-pump plotter class ready


In [17]:
hiscroll.enable_pump()

In [22]:
hiscroll.disable_pump()

In [12]:
%matplotlib notebook

In [ ]:
# Create and start live plotting
plotter = MultiPumpPlotter(update_interval=1000)  # Update every 2 seconds
plotter.start_live_plot()

# To stop plotting later, run: plotter.stop_live_plot()

In [23]:
plotter.stop_live_plot()

Live plotting stopped.


# 8. Disconnect

In [24]:
hiscroll.disconnect()
hipace_transfer.disconnect()
hipace_depo.disconnect()

2025-08-14 14:45:47 | INFO | Housekeeping stopped (internal mode)
2025-08-14 14:45:47 | INFO | Disconnected from Pfeiffer device hiscroll12
2025-08-14 14:45:48 | INFO | Housekeeping stopped (internal mode)
2025-08-14 14:45:48 | INFO | Disconnected from Pfeiffer device hipace_transfer
2025-08-14 14:45:49 | INFO | Housekeeping stopped (internal mode)
2025-08-14 14:45:49 | INFO | Disconnected from Pfeiffer device hipace_depo


True